<a href="https://colab.research.google.com/github/ShubhranshuPattnaik/Antenna-Fault-Detection-Dashboard/blob/main/Custom_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import tensorflow as tf
from transformers import DistilBertTokenizerFast, TFDistilBertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import gradio as gr

In [ ]:
df = pd.read_csv('/content/SMSSpamCollection.txt', sep='\t', names=["label", "message"])
X = list(df['message'])
y = list(df['label'])
y = list(pd.get_dummies(y, drop_first=True)['spam'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

In [ ]:
!pip install transformers

In [ ]:
from transformers import DistilBertTokenizerFast, TFDistilBertForSequenceClassification

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(X_train, truncation=True, padding=True, return_tensors='tf')
test_encodings = tokenizer(X_test, truncation=True, padding=True, return_tensors='tf')

In [ ]:
y_train = tf.convert_to_tensor(y_train)
y_test = tf.convert_to_tensor(y_test)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), y_train)).batch(8)
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), y_test)).batch(16)

In [ ]:
model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [ ]:
model.fit(train_dataset, epochs=2, batch_size=8)

In [ ]:
loss, accuracy = model.evaluate(test_dataset)
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

In [ ]:
predictions = model.predict(test_dataset).logits
predicted_classes = tf.argmax(predictions, axis=1)

In [ ]:
cm = confusion_matrix(y_test, predicted_classes)
print(cm)

In [ ]:

model.save_pretrained('senti_model')

In [ ]:
!pip install gradio

In [ ]:
def predict(text):
    encodings = tokenizer(text, truncation=True, padding=True, return_tensors='tf')
    predictions = model(encodings).logits
    predicted_class = tf.argmax(predictions, axis=1).numpy()[0]
    return "Spam" if predicted_class == 1 else "Not Spam"


interface = gr.Interface(fn=predict, inputs="text", outputs="text")


interface.launch()